In [1]:
import numpy as np
import pandas as pd

year = '2015'

In [2]:
payout={0:0., 1:0.008, 2:0.024, 3:0.05, 4:0.085, 5:0.13, 6:0.18}

In [3]:
sub = pd.read_csv('data/2015SubmissionRF.csv', index_col=0)
sub.head()

,Pred
id,
2015_1107_1112,0.121
2015_1107_1116,0.214
2015_1107_1124,0.169
2015_1107_1125,0.658
2015_1107_1129,0.316


In [4]:
preds = sub['Pred'].to_dict()

In [5]:
preds['2015_1458_1462']

0.882

In [6]:
playins = pd.read_csv('data/2015Playins.csv', index_col=0)
playins.head()

,Team,TeamID
0,Manhattan,1264
1,Hampton,1214
2,BYU,1140
3,Mississippi,1279
4,Dayton,1173


In [7]:
regs = pd.read_csv('data/2015Regulars.csv', index_col=0)
regs.tail()

,Team,TeamID
59,UAB,1412
60,Iowa,1234
61,Davidson,1172
62,Gonzaga,1211
63,N Dakota St,1295


In [8]:
teams = pd.read_csv('data/2015Teams.csv', index_col=0)
teams.tail()

,Team,TeamID
63,Mississippi,1279
64,Dayton,1173
65,Boise St,1129
66,North Florida,1316
67,Robert Morris,1352


In [9]:
index = dict(zip(teams['TeamID'], teams.index))

In [10]:
index[1462]

23

In [11]:
def predict_game(year, ID1, ID2, preds, rands):
    id1 = min(ID1, ID2)
    id2 = max(ID1, ID2)
    key = '{0}_{1}_{2}'.format(year, id1, id2)
    prob = preds[key]
    if prob > next(rands): # prob is probability ID1 wins
        return id1, id2
    else:
        return id2, id1

In [12]:
from numpy.random import random
teams15 = pd.concat([bracket15, playins15])
teams15 = teams15[np.array(['Playin' not in name for name in teams15['Team']])]
teams15 = teams15.reset_index(drop=True)
teams15
def sim_round(currentround, roundnum, year, preds, rands, res):
    nextround = []
    for i in range(0, len(currentround), 2):
        #print(teams.loc[index[currentround[i]], 'Team'], teams.loc[index[currentround[i+1]], 'Team'])
        winner, loser = predict_game(year, currentround[i], currentround[i+1], preds, rands)
        nextround.append(winner)
        res[index[loser]] = payout[roundnum] # roundnum = # wins
    return nextround

def sim_tourney(playins, regs, preds, year):
    res = np.zeros(68)
    rands = iter(random(68))
    pin = list(playins['TeamID'])
    
    # run playins
    for i, idx in enumerate(range(0, len(pin), 2)):
        winner, loser = predict_game(year, pin[idx], pin[idx+1], preds, rands)
        regs.loc[regs['Team'] == 'Playin'+str(i+1), 'TeamID'] = winner
    currentround = regs['TeamID']
    
    # run regular tournament
    for roundnum in range(6): # 6 rounds
        currentround = sim_round(currentround, roundnum, year, preds, rands, res)
    
    res[index[currentround[0]]] = payout[6] # tourney winner
    return res

In [13]:
res = sim_tourney(playins, regs, preds, year)
res

array([0.085, 0.008, 0.   , 0.024, 0.   , 0.008, 0.   , 0.   , 0.05 ,
       0.008, 0.   , 0.024, 0.   , 0.   , 0.008, 0.18 , 0.   , 0.   ,
       0.008, 0.008, 0.   , 0.024, 0.   , 0.024, 0.   , 0.008, 0.   ,
       0.008, 0.05 , 0.   , 0.085, 0.   , 0.008, 0.   , 0.   , 0.008,
       0.024, 0.   , 0.   , 0.05 , 0.   , 0.024, 0.   , 0.008, 0.   ,
       0.05 , 0.008, 0.   , 0.024, 0.   , 0.   , 0.008, 0.   , 0.008,
       0.024, 0.   , 0.008, 0.   , 0.13 , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.008, 0.   , 0.   , 0.   ])

In [14]:
res.sum()

1.0

In [15]:
res[index[1462]]

0.024

In [ ]:
%%time
Nsamples = 1000000
simtourneys = np.zeros((Nsamples, 68))
for i in range(Nsamples):
    simtourneys[i,:] = sim_tourney(playins, regs, preds, year)
np.save('data/2015simtourneys.npy', simtourneys)

In [25]:
simtourneys = np.load('data/2015simtourneys.npy')

In [26]:
myteams = np.zeros(68)
myteams[index[1462]] = 1
simresults = np.dot(simtourneys,myteams)
print("Mean = {0}, stdev = {1}".format(simresults.mean(), simresults.std()))

Mean = 0.01726, stdev = 0.01969889336993325


In [27]:
myteams2 = np.zeros(68)
myteams2[index[1459]] = 1
simresults = np.dot(simtourneys,myteams2)
print("Mean = {0}, stdev = {1}".format(simresults.mean(), simresults.std()))

Mean = 0.00131, stdev = 0.006157101590846136


In [28]:
totpool = 944.
def getID(row):
    try:
        ID = teamids.loc[teamids['TeamName'] == row['Team'], 'TeamID'].values[0]
    except:
        ID = 0
    return ID

teamids = pd.read_csv('data/Teams.csv')
jeff = ['Kansas', 'Notre Dame', 'Louisville', 'Maryland', 'Georgetown', 'Wichita St', 'Purdue', 'Oklahoma St', 'Davidson', 'Georgia', 'Texas', 'Buffalo']
jeff = pd.DataFrame(jeff, columns=['Team'])
jeff['TeamID'] = jeff.apply(getID, axis=1)
jeffteams = np.zeros(68)
for ID in jeff['TeamID']:
    jeffteams[index[ID]] = 1
    
simresults = np.dot(simtourneys, jeffteams)
print("Mean = {0}, stdev = {1}".format(simresults.mean()*totpool, simresults.std()*totpool))

Mean = 177.496544, stdev = 63.22450358973223


In [30]:
jeffteams = np.load('data/2015Jeff.npy')
simresults = np.dot(simtourneys, jeffteams)
print("Mean = {0}, stdev = {1}".format(simresults.mean()*totpool, simresults.std()*totpool))

Mean = 154.180815912, stdev = 58.09131600341862


In [33]:
field = np.load('data/2015field.npy')
simresults = np.dot(simtourneys, field)
print("Mean={0:.2f}. 10/30/50/70/90th percentile:".format(simresults.mean()), ["{0:.2f}".format(np.percentile(simresults, i)) for i in [10, 30, 50, 70, 90]])

Mean=0.04. 10/30/50/70/90th percentile: ['0.02', '0.02', '0.03', '0.05', '0.08']
